In [ ]:
# Copyright 2025 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### ADK Agent Engine 배포

In [ ]:
import os
from dotenv import load_dotenv
import argparse

import vertexai
from vertexai import agent_engines
from vertexai.agent_engines import AdkApp

from agent import root_agent

In [ ]:
# 환경 변수 로드.
load_dotenv()

from dotenv import dotenv_values
config = dotenv_values("../../.env")
for key, value in config.items():
    print(f"{key}: {value}")

#### Build ADK Agent.

In [ ]:
# Vertex AI 환경 초기화 : Agent Engine을 배포하기 위한 환경 설정
# Agent Engine 과 Gemini 모델을 사용하기 위한 리전을 다르게 할 수 있음.
vertexai.init(
    project=os.getenv("GOOGLE_CLOUD_PROJECT"),
    location=os.getenv("GOOGLE_CLOUD_LOCATION"),
    staging_bucket=os.getenv("AGENT_ENGINE_BUCKET"),
)

In [ ]:
agent_name = "engine_0122"

In [ ]:
# adk_app 생성
adk_app = AdkApp(agent=root_agent)

# Agent Engine 환경 설정
display_name = agent_name
gcs_dir_name = os.getenv("AGENT_ENGINE_BUCKET")
description = "사용자 질문에 대한 AI 정보 검색 어시스턴트"

requirements = [
    "google-adk[vertexai]",
    "google-cloud-aiplatform[adk,agent-engines]",
    "cloudpickle==3.0",
    "python-dotenv",
]

#### Deploy or Update an Agent Engine with ADK Framework.

In [ ]:
# Trace 와 Logging 설정 
env_vars = {
  "GOOGLE_CLOUD_AGENT_ENGINE_ENABLE_TELEMETRY": "true",
  "OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT": "true",
}

create = False

if create:
    # 최초 Agent Engine을 디플로이.
    remote_agent = agent_engines.create(
                    adk_app,
                    display_name=display_name,
                    gcs_dir_name = gcs_dir_name,
                    description=description,
                    requirements=requirements,
                    extra_packages = []
    )
else:
    # 이미 디플로이된 Agent를 업데이트.
    # 아래 Resource name은 Console 에서 확인 가능.
    agent_engine_resource_name = "projects/721521243942/locations/us-central1/reasoningEngines/449705753318522880"
    
    remote_agent = agent_engines.update(
        resource_name = agent_engine_resource_name,
        agent_engine = adk_app
    )    

#### Run Agent with Python Code

In [ ]:

agent_engine_resource_name = "projects/721521243942/locations/us-central1/reasoningEngines/449705753318522880"

remote_agent = agent_engines.get(agent_engine_resource_name)
print(f"Display name : {remote_agent.display_name}")
print(f"Reource name : {remote_agent.resource_name}")


remote_session = await remote_agent.async_create_session(user_id="u_456")
print(f"AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.")

while True:
    print("\n👤 User :   ")

    # 사용자 입력에서 쿼리 가져오기
    query = input("")
    if query.strip().lower() in ["exit", "quit"]:
        break
    else:
        print(query)

    events = remote_agent.async_stream_query(user_id="u_456", 
                                    session_id=remote_session["id"],
                                    message=query)
    
    async for event in events:
        response = event['content']['parts'][0]['text']
        print(f"\n 🤖 AI Assistant: {response}\n")


#### Run Agent with REST API(Python)

In [ ]:
import requests
import google.auth
import google.auth.transport.requests

credentials, project = google.auth.default()
auth_request = google.auth.transport.requests.Request()
credentials.refresh(auth_request)
token = credentials.token

region = os.getenv("GOOGLE_CLOUD_LOCATION")

resource_name = "projects/721521243942/locations/us-central1/reasoningEngines/449705753318522880"
url = f"https://{region}-aiplatform.googleapis.com/v1/{resource_name}:streamQuery?alt=sse"

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

print(f"AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.")

while True:
  print("\n👤 User :   ")

  # 사용자 입력에서 쿼리 가져오기
  query = input("")
  if query.strip().lower() in ["exit", "quit"]:
      break
  else:
      print(query)

  payload = {
      "class_method": "async_stream_query",
      "input": {
        "user_id": "u_456",
        "message": query,
      }
    }

  response = requests.post(url, json=payload, headers=headers)
  outcome = response.json()['content']['parts'][0]['text']

  print("\n 🤖 AI Assistant: ", outcome)